In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score, roc_curve,auc
tf.config.experimental_run_functions_eagerly(True)
RANDOM_SEED = 2021
TEST_PCT = 0.3


alpha_values = list()
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.beta = tf.Variable(0.1)

        self.encoding_dim = 14
        self.hidden_dim_1 = int(encoding_dim / 2)  #
        self.hidden_dim_2 = 4
        self.learning_rate = 1e-7

        self.input_layer = tf.keras.layers.Input(shape=(1,))
        self.d0 = tf.keras.layers.Dense(encoding_dim, activation=self.prelu,
                                        activity_regularizer=tf.keras.regularizers.l2(learning_rate))
        self.d1 = tf.keras.layers.Dropout(0.2)
        self.d2 = tf.keras.layers.Dense(hidden_dim_1, activation=self.prelu)
        self.d3 = tf.keras.layers.Dense(hidden_dim_2, activation=self.prelu)
        # Decoder
        self.d4 = tf.keras.layers.Dense(hidden_dim_1, activation=self.prelu)
        self.d5 = tf.keras.layers.Dropout(0.2)
        self.d6 = tf.keras.layers.Dense(encoding_dim, activation=self.prelu)
        self.d7 = tf.keras.layers.Dense(input_dim, activation=self.prelu)

    def prelu(self, x):
        ef=x/(1+np.absolute(x))
        return ef * self.beta

    def call(self, x, **kwargs):
        alpha_values.append(self.beta.numpy())

        x = self.d0(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)
        x = self.d5(x)
        x = self.d6(x)
        x = self.d7(x)

        return x

if __name__ == '__main__':
    data = pd.read_csv('yahoo1.csv')
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values

    X_train = []

    normal_data = X[y == 0]
    train_threshold = math.floor(0.7 * len(data))

    for i in range(train_threshold):
        X_train.append(normal_data[i])

    nb_epoch = 50
    batch_size = 64
    input_dim = X.shape[1]  # num of columns, 30
    encoding_dim = 14
    hidden_dim_1 = int(encoding_dim / 2)  #
    hidden_dim_2 = 4
    learning_rate = 1e-7
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    encoder = tf.keras.layers.Dense(encoding_dim, activation="tanh",
                                    activity_regularizer=tf.keras.regularizers.l2(learning_rate))(input_layer)
    encoder = tf.keras.layers.Dropout(0.2)(encoder)
    encoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(hidden_dim_2, activation=tf.nn.leaky_relu)(encoder)
    # Decoder
    decoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
    decoder = tf.keras.layers.Dropout(0.2)(decoder)
    decoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(input_dim, activation='tanh')(decoder)
    autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
    autoencoder.summary()
    # define our early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0001,
        patience=10,
        verbose=1,
        mode='min',
        restore_best_weights=True)
    modelq10 = MyModel()
    modelq10.compile(metrics=['accuracy'],loss='mean_squared_error', optimizer='adam')
    X_test = X
    X_train = np.array(X_train)
    history=modelq10.fit(X_train, X_train,
                              epochs=nb_epoch,
                              batch_size=batch_size,
                              shuffle=True,
                              validation_data=(X_test, X_test),
                              verbose=1,
                              callbacks=[ early_stop]
                              ).history

    plt.plot(history['loss'], linewidth=2, label='Train')
    plt.plot(history['val_loss'], linewidth=2, label='Test')
    plt.legend(loc='upper right')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    test_x_predictions = modelq10.predict(X_test)
    mse = np.mean(np.power(X_test - test_x_predictions, 2), axis=1)
    print('mse',mse)
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
    ax1.set_title('comparison')
    ax1.set_xlabel('time')
    ax1.set_ylabel('mse')
    ax1.plot(mse, label="dataset")

    ax2.set_xlabel('time')
    ax2.set_ylabel('value')
    ax2.plot(X_test, label="test_data")
    plt.legend(loc='best')
    plt.show()
    mean_mse=np.mean(mse)
    std_mse=np.std(mse)
    lper = np.percentile(mse, 5)
    uper = np.percentile(mse, 95)
    print('lper', lper)
    print('uper', uper)
    y_pred = mse.copy()
    y_pred = np.array(y_pred)

    count = 0
    for itr in range(len(y_pred)):
        if (mse[itr] <= lper or mse[itr] >= uper):
            count = count + 1
            y_pred[itr]=1
        else:
            y_pred[itr]=0

    precision_auto = precision_score(y, y_pred)
    recall_auto = recall_score(y, y_pred)
    f1_score_auto = f1_score(y, y_pred)
    fpr8, tpr8, thresholds = roc_curve(y, y_pred)
    auc_roc_auto = auc(fpr8, tpr8)


    print("Precision: {:.4f}".format(precision_auto))
    print("Recall: {:.4f}".format(recall_auto))
    print("F1-score: {:.4f}".format(f1_score_auto))
    print("AUC-ROC: {:.4f}".format(auc_roc_auto))